In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import sys
import os
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if path not in sys.path:
    sys.path.append(path)
import pandas as pd
from utils.helpers import is_question_arabic
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import joblib

c:\Users\nasser\.conda\envs\finall\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv('../data/raw.csv')

In [3]:
df = df[['message']]

In [4]:
# remove null values
df = df.dropna()

# Label data using Regex

In [5]:
df['is_question'] = df['message'].apply(lambda x : 1 if is_question_arabic(x) else 0)

# Train NN model

In [6]:
messages = df['message'].values
labels = df['is_question'].values
vocab_size = 10000 # Choose a vocabulary size
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(messages)
sequences = tokenizer.texts_to_sequences(messages)
X = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=100)
y = tf.keras.utils.to_categorical(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Define RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [8]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
# Train model and prevent overfitting with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=20, batch_size=2048, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/20
25/25 [==============================] - 6s 42ms/step - loss: 0.6629 - accuracy: 0.5987 - val_loss: 0.6158 - val_accuracy: 0.6452
Epoch 2/20
25/25 [==============================] - 1s 26ms/step - loss: 0.5108 - accuracy: 0.7669 - val_loss: 0.3302 - val_accuracy: 0.8624
Epoch 3/20
25/25 [==============================] - 1s 27ms/step - loss: 0.2426 - accuracy: 0.9135 - val_loss: 0.1527 - val_accuracy: 0.9473
Epoch 4/20
25/25 [==============================] - 1s 26ms/step - loss: 0.1160 - accuracy: 0.9663 - val_loss: 0.1338 - val_accuracy: 0.9536
Epoch 5/20
25/25 [==============================] - 1s 27ms/step - loss: 0.0801 - accuracy: 0.9795 - val_loss: 0.1080 - val_accuracy: 0.9664
Epoch 6/20
25/25 [==============================] - 1s 26ms/step - loss: 0.0658 - accuracy: 0.9836 - val_loss: 0.1319 - val_accuracy: 0.9646
Epoch 7/20
25/25 [==============================] - 1s 27ms/step - loss: 0.0742 - accuracy: 0.9810 - val_loss: 0.1061 - val_accuracy: 0.9675
Epoch 8/20
25

In [10]:
# Evaluate model on test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss on test set: {loss}")
print(f"Accuracy on test set: {accuracy}")

# Save model and tokenizer
model.save('..\models\question_classifier.h5')
joblib.dump(tokenizer, '..\models\question_tokenizer.joblib')

499/499 [==============================] - 3s 5ms/step - loss: 0.1125 - accuracy: 0.9655
Loss on test set: 0.11254055798053741
Accuracy on test set: 0.9655020833015442


['..\\models\\question_tokenizer.joblib']

In [11]:
# create dataframa with text and predict if it is a question or not
new_df = pd.DataFrame({'message': ['شو الاخبار ']})
# add more rows

# tokenize the text
new_sequences = tokenizer.texts_to_sequences(new_df['message'].values)
# pad the sequences
new_X = tf.keras.preprocessing.sequence.pad_sequences(new_sequences, maxlen=100)
# predict
model.predict(new_X)


1/1 [==============================] - 0s 259ms/step


array([[0.00298598, 0.99701405]], dtype=float32)